In [1]:
import os
currentDirectory = os.getcwd()
data_dir = currentDirectory[:-7] + '/input/'
print(data_dir)

from xmitgcm import open_mdsdataset
ds1 = open_mdsdataset(data_dir, geometry='cartesian', endian='<',prefix=['statevars'])

/scratch/jxchang/TallRid/results/TideU008sinN0012H200ho140Ah0200Kh0200_freeslipBotSide_Cdqdt001/input/


In [2]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from matplotlib.colors import BoundaryNorm
import matplotlib.animation as animation
import math

In [21]:
t = 0
h0 = 140.

f0 = 1.e-4
g = 9.8
u0=0.08

om=2*np.pi/12.4/3600 
alpha = 2e-4 

tR_fname="../indata/TRef.bin"
tRef = np.fromfile(tR_fname)
refSalt=35.
refTemp=tRef[0]
print('refTemp='+ str(refTemp))
print('tRef='+ str(tRef))

xmin = -4
xmax = 4
vmax = 1
vmin = -1
wmax = 0.2
wmin = -0.2
tmax = math.floor(np.amax(tRef))
tmin = math.floor(np.amin(tRef))
numcolt=21
numcolv=30

ttlen=len(ds1.time)
print('the length of time:' + str(ttlen) )
print(tmin)

refTemp=34.63265306122449
tRef=[34.63265306 34.26530612 33.89795918 33.53061224 33.16326531 32.79591837
 32.42857143 32.06122449 31.69387755 31.32653061 30.95918367 30.59183673
 30.2244898  29.85714286 29.48979592 29.12244898 28.75510204 28.3877551
 28.02040816 27.65306122 27.28571429 26.91836735 26.55102041 26.18367347
 25.81632653 25.44897959 25.08163265 24.71428571 24.34693878 23.97959184
 23.6122449  23.24489796 22.87755102 22.51020408 22.14285714 21.7755102
 21.40816327 21.04081633 20.67346939 20.30612245]
the length of time:293
20


In [22]:
xg=ds1.coords["XG"]
xc=ds1.coords["XC"]
yg=ds1.coords["YG"]
yc=ds1.coords["YC"]
z=ds1.coords["Z"]
time=ds1.coords['time'].values/np.timedelta64(1, 's')/3600
ds1=ds1.assign_coords(XC0 = (ds1.XC/1000.-ds1.XC.mean()/1000.))
ds1=ds1.assign_coords(XG0 = (ds1.XG-ds1.XC.mean())/1000.)
ds1=ds1.assign_coords(YC0 = (ds1.YC/1000.-ds1.YC.mean()/1000.))
ds1=ds1.assign_coords(YG0 = (ds1.YG-ds1.YC.mean())/1000.)
yc0 = (ds1.YC/1000.-ds1.YC.mean()/1000.)
xc0 = (ds1.XC/1000.-ds1.XC.mean()/1000.)

In [23]:
fig = plt.figure(figsize=(11,8))
grid = plt.GridSpec(2, 2, height_ratios=[0.6,2.5], width_ratios=[1,0.1],wspace=0.02,hspace=0.38)

<Figure size 792x576 with 0 Axes>

In [24]:
SMALL_SIZE = 17
MEDIUM_SIZE = 19
BIGGER_SIZE = 21

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [25]:
#plot bathymetry
ax1=fig.add_subplot(grid[0,:-1])
ax1.plot(time,np.zeros(ttlen),color='gray',linestyle='--')
ax1.plot(time,ds1.UVEL.isel(Z=0,YC=0,XG=0),label='U(t,0,0,0)')
ax1.plot(time,u0*np.sin(om*ds1.time.values/np.timedelta64(1, 's')),label='Uinput')
ax1.set_xlim(min(time), max(time))
ax1.set_ylim(-u0-0.05, u0+0.05)
point, = ax1.plot([time[0]],[ds1.UVEL.isel(time=t,XG=0,Z=0).data], 'go', color='red',ms=10)
ax1.set_xlabel('time [hr]')
ax1.set_ylabel('U [m/s]')
ax1.set_title('Uinput vs U[t,0,0,0] ', loc='left')



Text(0.0, 1.0, 'Uinput vs U[t,0,0,0] ')

In [26]:
# plot temperature
ax2=fig.add_subplot(grid[1,0])
cmap = cm.RdBu_r
levels = np.linspace(vmin, vmax, num=numcolv)
Tlevels = np.linspace(tmin,tmax,7)
print(Tlevels)

norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
opts = {'cmap':cm.get_cmap(cmap, len(levels) - 1), 'norm': norm}

cset1 = ax2.contourf(ds1.XG0, ds1.Z, np.squeeze(ds1.UVEL.isel(time=t)), levels , extend='max', **opts)
cset2 = ax2.contour(ds1.XC0, ds1.Z, np.squeeze(ds1.THETA.isel(time=t)), np.sort(tRef[::4]) , color='Gray')
ax2.clabel(cset2, fontsize=9, inline=1)

cax = fig.add_subplot(grid[1,-1])
cax.set_visible(False)
fig.colorbar(cset1, ax=cax,extend='both')
#fig.colorbar(cset1, ax=axs,ticks=np.arange(tmin, tmax+0.1, 5))
ax2.set_xlabel('X [km]')
ax2.set_ylabel('Z [m]')
ax2.set_title('T [oC]', loc='left')
ax2.set(xlim=(xmin, xmax),ylim=(-208,0))
ax2.plot(ds1['XC0'],-np.squeeze(ds1.Depth),lw=1.5)

[20.         22.33333333 24.66666667 27.         29.33333333 31.66666667
 34.        ]


/home/jxchang/jupyter_py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: The following kwargs were not used by contour: 'color'
  if sys.path[0] == '':
/home/jxchang/jupyter_py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:17: MatplotlibDeprecationWarning: The 'extend' parameter to Colorbar has no effect because it is overridden by the mappable; it is deprecated since 3.3 and will be removed two minor releases later.


In [27]:
fig
print(ax2.collections)

[<matplotlib.collections.PathCollection object at 0x2b9066a3f6d0>, <matplotlib.collections.PathCollection object at 0x2b9066fbcad0>, <matplotlib.collections.PathCollection object at 0x2b9066fbccd0>, <matplotlib.collections.PathCollection object at 0x2b9066fbc390>, <matplotlib.collections.PathCollection object at 0x2b9066fbc790>, <matplotlib.collections.PathCollection object at 0x2b9066c4f050>, <matplotlib.collections.PathCollection object at 0x2b9066c4ff50>, <matplotlib.collections.PathCollection object at 0x2b90650f5cd0>, <matplotlib.collections.PathCollection object at 0x2b90650c6e10>, <matplotlib.collections.PathCollection object at 0x2b9066c4fc50>, <matplotlib.collections.PathCollection object at 0x2b9065089610>, <matplotlib.collections.PathCollection object at 0x2b9065089c50>, <matplotlib.collections.PathCollection object at 0x2b9065089e50>, <matplotlib.collections.PathCollection object at 0x2b9066b57750>, <matplotlib.collections.PathCollection object at 0x2b9066b57b90>, <matplotl

In [28]:
tt: int=0
def updateData(tt):

    updatetime = time[tt]
    print(tt)
    ax2.collections = []
    ax2.contourf(ds1.XG0, ds1.Z, np.squeeze(ds1.UVEL.isel(time=tt)), levels, extend='max', **opts)
    ax2.contour(ds1.XC0, ds1.Z, np.squeeze(ds1.THETA.isel(time=tt)), np.sort(tRef[::4]), color='Gray')
    point.set_data(updatetime,ds1.UVEL.isel(time=tt,XG=0,Z=0))
    ax1.set_title('t= %2.3fhr' % updatetime)
    fig.savefig('./figsMag2/summary_TU_xz_t%s_x%s.png' %(tt,xmax))



simulation = animation.FuncAnimation(fig, updateData, blit=False, frames= 168 , interval=40, repeat=False)
plt.draw()
plt.show()
simulation.save(filename='summary_TU_xz_x%s.gif' %xmax,writer='imagemagick', fps=2)

<Figure size 432x288 with 0 Axes>

0


/home/jxchang/jupyter_py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: The following kwargs were not used by contour: 'color'
  


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167


In [29]:
from IPython.display import Image
from IPython.display import display
display(Image(url='./summary_TU_xz_x60.gif'))